In [1]:
import os
os.chdir('../')
os.getcwd()

'/home/izam/coding/Customer-Churn'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PredictionConfig:
    root_dir: str
    model_path: str
    vectorizer_path: str
    data_path: str
    prediction_file: str
    target_column: str

In [3]:
from CustomerChurn.constants import *
from CustomerChurn.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_prediction_config(self) -> PredictionConfig:
        config = self.config.prediction
        target_column = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        prediction_config = PredictionConfig(
            root_dir = config.root_dir,
            model_path= config.model_path,
            vectorizer_path=config.vectorizer_path,
            data_path=config.data_path,
            prediction_file=config.prediction_file,
            target_column=target_column.name,
           
        )

        return prediction_config

In [5]:
from CustomerChurn import logger
from CustomerChurn.utils.common import load_bin, save_json, save_txt, get_size
import pandas as pd
import numpy as np
import urllib.request as request

In [6]:
class Prediction:
    def __init__(self, config: PredictionConfig):
        self.config = config

    def _download_file(self, url, datapath):
            if not os.path.exists(datapath):
                filename, headers = request.urlretrieve(
                    url = url,
                    filename = datapath
                )
                logger.info(f"{filename} download!")
                save_txt(data=str(headers), path=Path(os.path.join(self.config.root_dir, "download_status.txt")))
            else:
                logger.info(f"File already exists of size: {get_size(Path(datapath))}")

    def predict(self):
        model = load_bin(Path(self.config.model_path))
        vectorizer = load_bin(Path(self.config.vectorizer_path))
        try:
            data = pd.read_csv(self.config.data_path)
        except Exception as e:
            logger.info(f"error - {e} while access predict data")
            self._download_file("https://raw.githubusercontent.com/izam-mohammed/data-source/main/sample_customer_churn.csv", os.path.join(self.config.root_dir, "sample_data.csv"))
            data = pd.read_csv(os.path.join(self.config.root_dir, "sample_data.csv"))

        data[self.config.target_column] = np.array(["Yes"])
        

        vectorized_data = vectorizer.transform(data)
        vectorized_data = vectorized_data[:,:-1]
        prediction = model.predict(vectorized_data)
        logger.info(f"predicted the new data as {prediction[0]}")

        save_json(path=self.config.prediction_file, data={'prediction':float(prediction[0])})

In [7]:
try:
    config = ConfigurationManager()
    prediction_config = config.get_prediction_config()
    prediction_config = Prediction(config=prediction_config)
    prediction_config.predict()
except Exception as e:
    raise e

[2023-12-16 09:42:13,030: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-16 09:42:13,034: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-16 09:42:13,040: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-16 09:42:13,042: INFO: common: created directory at: artifacts]
[2023-12-16 09:42:13,042: INFO: common: created directory at: artifacts/prediction]
[2023-12-16 09:42:13,495: INFO: common: binary file loaded from: artifacts/model_trainer/model.joblib]
[2023-12-16 09:42:13,532: INFO: common: binary file loaded from: artifacts/data_transformation/encoder.joblib]
[2023-12-16 09:42:13,534: INFO: 1249834910: error - [Errno 2] No such file or directory: 'artifacts/prediction/data.csv' while access predict data]
[2023-12-16 09:42:13,895: INFO: 1249834910: artifacts/prediction/sample_data.csv download!]
[2023-12-16 09:42:13,897: INFO: common: txt file saved at: artifacts/prediction/download_status.txt]
[2023-12-16 09:42:13,909: INF